In [2]:
import rdflib
from nanopub import Publication, NanopubClient

## Using blank nodes within assertions
Upon publication, any blank nodes in the rdf graph are replaced with the nanopub's URI, with the blank node name as a
fragment. For example, if the blank node is called 'step', that would result in a URI composed of the nanopub's (base)
URI, followed by #step. We can thus use blank nodes to refer to new concepts, making use of the namespace of the 
to-be-published URI.

In [23]:
my_assertion = rdflib.Graph()

# We want to introduce a new concept in our publication: Tim Berners Lee
tim = rdflib.BNode('timbernerslee')

# We assert that he is a person
my_assertion.add((tim, rdflib.RDF.type, rdflib.FOAF.Person) )

# And create a publication object for this assertion
publication = Publication.from_assertion(assertion_rdf=my_assertion)

In [24]:
# The assertion now uses a dummy namespace (http://purl.org/nanopub/temp/mynanopub#) that will later be replaced with the nanopub uri
print(publication.assertion.serialize(format='trig').decode())

@prefix : <http://purl.org/nanopub/temp/mynanopub#> .

:assertion {
    :timbernerslee a <http://xmlns.com/foaf/0.1/Person> .
}




In [25]:
# Let's publish this to the test server
client = NanopubClient(use_test_server=True)
pubinfo = client.publish(publication)

Published to http://purl.org/np/RA_j6TPcnoQJ_XkISjugTgaRsFGLhpbZCC3mE7fXs0REI


In [31]:
# Let's fetch the freshly published nanopub:
published = client.fetch(pubinfo['nanopub_uri'])

# As you see the dummy namespace has been replaced with the nanopub URI namespace!
print(published.assertion.serialize(format='trig').decode())

@prefix sub: <http://purl.org/np/RA_j6TPcnoQJ_XkISjugTgaRsFGLhpbZCC3mE7fXs0REI#> .

sub:assertion {
    sub:timbernerslee a <http://xmlns.com/foaf/0.1/Person> .
}




## Introducing a concept
You can optionally specify that the ```Publication``` introduces a particular concept using blank nodes. 
The pubinfo graph will note that this nanopub npx:introduces the concept. The concept should be a blank node 
(rdflib.term.BNode), and is converted to a URI derived from the nanopub's URI with a fragment (#) made from the blank
node's name.

In [32]:
# We can create a publication introducing this new concept
publication = Publication.from_assertion(assertion_rdf=my_assertion,
                                         introduces_concept=tim)

In [33]:
# Let's publish this to the test server
client = NanopubClient(use_test_server=True)
pubinfo = client.publish(publication)

Published to http://purl.org/np/RAuB_kEHoclqVRFSip1dB1yYirdx-TasAT44kBvGy6gQ0
Published concept to http://purl.org/np/RAuB_kEHoclqVRFSip1dB1yYirdx-TasAT44kBvGy6gQ0#timbernerslee


In [34]:
# pubinfo now contains the concept uri
pubinfo['concept_uri']

'http://purl.org/np/RAuB_kEHoclqVRFSip1dB1yYirdx-TasAT44kBvGy6gQ0#timbernerslee'

In [35]:
# The publication info subgraph notes that this nanopub introduces Tim Berners Lee
print(publication.pubinfo.serialize(format='trig').decode())

@prefix : <http://purl.org/nanopub/temp/mynanopub#> .
@prefix npx: <http://purl.org/nanopub/x/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:pubInfo {
    : npx:introduces :timbernerslee ;
        prov:generatedAtTime "2020-11-24T11:24:53.959647"^^xsd:dateTime ;
        prov:wasAttributedTo <https://orcid.org/0000-0000-0000-0000> .
}


